# 고장시점 선형예측

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
%config InlineBackend.figure_format = 'retina'

from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection #import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn import metrics  # mean_squared_error, mean_absolute_error, median_absolute_error, explained_variance_score, r2_score
from sklearn.feature_selection import SelectFromModel, RFECV

# 데이터

In [4]:
df_train = pd.read_csv('PM_train_pre.csv')
df_train.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,ttf,label_bnc,label_mcc
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,0,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,0,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,0,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,0,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,0,0


In [5]:
df_test = pd.read_csv('PM_test_pre.csv')
df_test.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,ttf,label_bnc,label_mcc
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,8.4052,0.03,392,2388,100.0,38.86,23.3735,142,0,0
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,8.3803,0.03,393,2388,100.0,39.02,23.3916,141,0,0
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,8.4441,0.03,393,2388,100.0,39.08,23.4166,140,0,0
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,8.3917,0.03,391,2388,100.0,39.00,23.3737,139,0,0
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,8.4031,0.03,390,2388,100.0,38.99,23.4130,138,0,0


In [41]:
features = ['setting1','setting2','setting3','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21']
features_sensors = ['s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21']

X_train = df_train[features]
y_train = df_train['ttf']

X_test = df_test[features]
y_test = df_test['ttf']

# 회귀분석 성능 지표

In [7]:
def get_regression_metrics(model, actual, predicted):
    regr_metrics = {
      'Root Mean Squared Error' : metrics.mean_squared_error(actual, predicted)**0.5,
      'Mean Absolute Error' : metrics.mean_absolute_error(actual, predicted),
      'R^2' : metrics.r2_score(actual, predicted)}

    df_regr_metrics = pd.DataFrame.from_dict(regr_metrics, orient='index')
    df_regr_metrics.columns = [model]
    return df_regr_metrics

## 상관 계수와 중요 특성값

In [8]:
def plot_features_weights(model, weights, feature_names, weights_type='c'):
    (px, py) = (8, 10) if len(weights) > 30 else (8, 5)
    W = pd.DataFrame({'Weights':weights}, feature_names)
    W.sort_values(by='Weights', ascending=True).plot(kind='barh', color='r', figsize=(px,py))
    label = ' Coefficients' if weights_type =='c' else ' Features Importance'
    plt.xlabel(model + label)
    plt.gca().legend_ = None

## 잔차를 그리는 함수

In [9]:
def plot_residual(model, y_train, y_train_pred, y_test, y_test_pred):
    plt.scatter(y_train_pred, y_train_pred - y_train, c='blue', marker='o', label='Training data', s=2)
    plt.scatter(y_test_pred, y_test_pred - y_test, c='lightgreen', marker='s', label='Test data', s=2)
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.legend(loc='upper left')
    plt.hlines(y=0, xmin=-50, xmax=400, color='red', lw=2)
    plt.title(model + ' Residuals')
    plt.show()

# 회귀 분석
## 선형 모델

In [10]:
linreg = linear_model.LinearRegression()
linreg.fit(X_train, y_train)

y_test_predict = linreg.predict(X_test)
y_train_predict = linreg.predict(X_train)

print('R^2 training: %.3f, R^2 test: %.3f' % (
      (metrics.r2_score(y_train, y_train_predict)), 
      (metrics.r2_score(y_test, y_test_predict))))

linreg_metrics = get_regression_metrics('Linear Regression', y_test, y_test_predict)
linreg_metrics


R^2 training: 0.580, R^2 test: 0.327


,Linear Regression
Root Mean Squared Error,48.399484
Mean Absolute Error,37.173907
R^2,0.326553


In [11]:
# plot_features_weights('Linear Regression', linreg.coef_, X_train.columns, 'c')

In [12]:
# plot_residual('Linear Regression', y_train_predict, y_train, y_test_predict, y_test)

## 다차항 모델

In [13]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)

polyreg = linear_model.LinearRegression()
polyreg.fit(X_train_poly, y_train)

y_test_predict = polyreg.predict(X_test_poly)
y_train_predict = polyreg.predict(X_train_poly)

print('R^2 training: %.3f, R^2 test: %.3f' % (
      (metrics.r2_score(y_train, y_train_predict)), 
      (metrics.r2_score(y_test, y_test_predict))))

polyreg_metrics = get_regression_metrics('Polynomial Regression', y_test, y_test_predict)
polyreg_metrics

R^2 training: 0.627, R^2 test: 0.373


,Polynomial Regression
Root Mean Squared Error,46.703581
Mean Absolute Error,35.276368
R^2,0.372921


In [14]:
X_train.shape, X_train_poly.shape

((20631, 24), (20631, 325))

In [15]:
# plot_residual('Polynomial Regression', y_train_predict, y_train, y_test_predict, y_test)

- Polynomial 회귀의 성능이 선형 모델보다 더 낫다

## 랜덤 포레스트

In [16]:
rf = RandomForestRegressor(n_estimators=100, max_features=3, max_depth=4) # original features

rf.fit(X_train, y_train)

y_test_predict = rf.predict(X_test)
y_train_predict = rf.predict(X_train)

print('R^2 training: %.3f, R^2 test: %.3f' % (
      (metrics.r2_score(y_train, y_train_predict)), 
      (metrics.r2_score(y_test, y_test_predict))))

rf_metrics = get_regression_metrics('Random Forest Regression', y_test, y_test_predict)
rf_metrics

R^2 training: 0.595, R^2 test: 0.318


,Random Forest Regression
Root Mean Squared Error,48.718634
Mean Absolute Error,37.201166
R^2,0.317642


In [17]:
# plot_features_weights('Random Forest Regressor', rf.feature_importances_, X_train.columns, 't' )

In [18]:
# plot_residual('Random Forest Regression', y_train_predict, y_train, y_test_predict, y_test)

### 실제값과 예측치 보기

In [19]:
rf_pred_dict = {
    'Actual' : y_test,
    'Prediction' : y_test_predict }
    
rf_pred = pd.DataFrame.from_dict(rf_pred_dict).T
rf_pred[:20].T

,Actual,Prediction
0,142.0,149.182787
1,141.0,150.801239
2,140.0,145.340816
3,139.0,148.399648
4,138.0,160.358962
...,...,...
13091,24.0,45.603753
13092,23.0,59.252196
13093,22.0,70.420720
13094,21.0,60.825239


In [30]:
X_test[:3]

,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,21.61,553.90,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,21.61,554.85,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,21.61,554.11,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166


In [54]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[features_sensors] = scaler.fit_transform(X_train[features_sensors])
X_test[features_sensors] = scaler.transform(X_test[features_sensors])

/Users/hwa-kim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/hwa-kim/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/hwa-kim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [55]:
linreg = linear_model.LinearRegression()
linreg.fit(X_train, y_train)

y_test_predict = linreg.predict(X_test)
metrics.r2_score(y_test, y_test_predict)

0.3265530194595667

In [58]:
X_train.shape

(20631, 24)

In [ ]:
# !pip install tensorflow
# !pip install keras

In [62]:
from keras import Sequential
from keras.layers import Dense

model = Sequential()
# model.add(Dense(300, activation='relu', input_dim=24)) 
model.add(Dense(300, activation='relu', input_shape=(24,)) 
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(1))

In [60]:
import keras

In [63]:
model.compile(loss='mse',optimizer='adam')
history = model.fit(X_train, y_train, batch_size=20, epochs=20,
                validation_split=0.2, verbose=1,
callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

Train on 16504 samples, validate on 4127 samples
Epoch 1/20
16504/16504 [==============================] - 7s 405us/step - loss: 1938.9327 - val_loss: 2549.8473
Epoch 2/20
16504/16504 [==============================] - 7s 405us/step - loss: 1591.5018 - val_loss: 2655.1156
Epoch 3/20
16504/16504 [==============================] - 6s 382us/step - loss: 1560.1203 - val_loss: 2434.9448
Epoch 4/20
16504/16504 [==============================] - 6s 392us/step - loss: 1543.3188 - val_loss: 2557.2571
Epoch 5/20
16504/16504 [==============================] - 6s 386us/step - loss: 1521.1197 - val_loss: 2558.5401
Epoch 6/20
16504/16504 [==============================] - 6s 378us/step - loss: 1519.1213 - val_loss: 2515.8558
Epoch 7/20
16504/16504 [==============================] - 6s 384us/step - loss: 1505.0464 - val_loss: 2911.5890
Epoch 8/20
16504/16504 [==============================] - 7s 395us/step - loss: 1507.2990 - val_loss: 2604.0634


In [64]:
y_pred = model.predict(X_test)
metrics.r2_score(y_test, y_pred)

0.4086621368135249

In [65]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=24)) 
model.add(Dense(100, activation='relu'))
model.add(Dense(1))

In [69]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 100)               2500      
_________________________________________________________________
dense_13 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 101       
Total params: 12,701
Trainable params: 12,701
Non-trainable params: 0
_________________________________________________________________


In [66]:
model.compile(loss='mse',optimizer='adam')
history = model.fit(X_train, y_train, batch_size=20, epochs=20,
                validation_split=0.2, verbose=1,
callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

Train on 16504 samples, validate on 4127 samples
Epoch 1/20
16504/16504 [==============================] - 3s 195us/step - loss: 2240.6450 - val_loss: 2776.3034
Epoch 2/20
16504/16504 [==============================] - 2s 144us/step - loss: 1610.5961 - val_loss: 2607.7116
Epoch 3/20
16504/16504 [==============================] - 2s 148us/step - loss: 1581.8080 - val_loss: 2570.3569
Epoch 4/20
16504/16504 [==============================] - 2s 146us/step - loss: 1568.1281 - val_loss: 2579.6127
Epoch 5/20
16504/16504 [==============================] - 2s 144us/step - loss: 1554.5609 - val_loss: 2822.8091
Epoch 6/20
16504/16504 [==============================] - 3s 164us/step - loss: 1533.0707 - val_loss: 2707.0708
Epoch 7/20
16504/16504 [==============================] - 3s 159us/step - loss: 1527.6644 - val_loss: 2701.9640
Epoch 8/20
16504/16504 [==============================] - 3s 154us/step - loss: 1510.3629 - val_loss: 2635.0914


In [67]:
y_pred = model.predict(X_test)
metrics.r2_score(y_test, y_pred)

0.3906243080667574